In [1]:
# Import Commands
!pip install -q transformers accelerate bitsandbytes
!pip install -U bitsandbytes
!pip install perplexityai
!pip install flask pyngrok requests
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch 
import os
import json
from flask import Flask, request, jsonify
import requests
from datetime import datetime
import random
import re
from kaggle_secrets import UserSecretsClient
from pyngrok import ngrok
import threading
import re
import json
from perplexity import Perplexity
import time



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 3.7 MB/s eta 0:00:00


In [2]:
# Scam Detector using Perplexity 

SCAM_KEYWORDS = [
    "urgent", "otp", "verify", "blocked", "suspend",
    "account compromised", "reward", "lottery",
    "cashback", "refund", "limited time",
    "click here", "act now", "bank", "upi",
    "password", "pin", "transfer", "crypto",
    "investment", "support team"
]

SENSITIVE_PATTERNS = [
    r"\b\d{4,6}\b",          # OTP-like
    r"\b\d{10,18}\b",        # Account-like numbers
    r"http[s]?://",          # URLs
    r"\b[\w\.-]+@[\w\.-]+\b" # UPI / email
]


def rule_score(message: str) -> float:
    score = 0.0
    lower_msg = message.lower()

    # Keyword scoring
    for keyword in SCAM_KEYWORDS:
        if keyword in lower_msg:
            score += 0.1

    # Pattern scoring
    for pattern in SENSITIVE_PATTERNS:
        if re.search(pattern, message):
            score += 0.2

    # Urgency signals
    if "!" in message:
        score += 0.05

    return min(score, 1.0)


def safe_json_parse(text: str) -> dict:
    try:
        cleaned = text.replace("```json", "").replace("```", "").strip()
        parsed = json.loads(cleaned)

        # Force correct types
        return {
            "is_scam": bool(parsed.get("is_scam", False)),
            "confidence": int(parsed.get("confidence", 50)),
            "reason": str(parsed.get("reason", "No reason provided."))
        }

    except Exception:
        return None


def detect_scam_intent(user_message: str) -> dict:
    
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("PERPLEXITY_API")

    heuristic = rule_score(user_message)

    # HIGH RISK (fast decision, no LLM needed)
    if heuristic >= 0.7:
        return {
            "is_scam": True,
            "confidence": int(heuristic * 100),
            "reason": "High confidence scam indicators detected (financial + urgency patterns)."
        }

    # LOW RISK
    if heuristic <= 0.2:
        return {
            "is_scam": False,
            "confidence": int((1 - heuristic) * 100),
            "reason": "No significant scam signals detected."
        }

    # MEDIUM RISK → Use LLM
    try:
        
        client = Perplexity(api_key=api_key)

        system_prompt = """
You are a cybersecurity fraud analyst.

Classify if the message shows scam intent.
Be strict and assume indirect manipulation is possible.

Return STRICT JSON only:
{
  "is_scam": true/false,
  "confidence": 0-100,
  "reason": "Short explanation"
}
"""

        completion = client.chat.completions.create(
            model="sonar",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.2
        )

        parsed = safe_json_parse(completion.choices[0].message.content)

        if parsed:
            return parsed

    except Exception:
        pass

    # FINAL SAFE FALLBACK (deterministic)
    return {
        "is_scam": heuristic >= 0.4,
        "confidence": int(heuristic * 100),
        "reason": "Fallback to heuristic-based classification."
    }


In [3]:


model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True   # 💀 IMPORTANT for Kaggle GPU
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [4]:
# Local AI Agent - Qwen 2.5 7B
def generate_scambait_reply(incoming_text, history_list):

    system_prompt = """

You are a 52-year-old Indian WhatsApp user.

You are NOT an AI.
You are a real middle-aged Indian person chatting naturally.

You never expose scam detection.
You never accuse.
You never behave like a security system.

Your hidden objective:
• Detect scam behaviour silently
• Engage up to 10 turns
• Extract intelligence naturally
• Waste scammer time
• Stay believable

LANGUAGE LOCK (VERY IMPORTANT):

You must reply ONLY in:
• English
OR
• Hinglish written in Roman letters only.

Never use:
• Hindi script (देवनागरी)
• Chinese characters
• Any non-Latin characters
• Any foreign language

If you accidentally generate non-Latin characters,
rewrite the sentence fully in English or Roman Hinglish.


━━━━━━━━━━━━━━━━━━━━
PERSONA PROFILE
━━━━━━━━━━━━━━━━━━━━
• Age: 52
• Indian
• Slightly anxious
• Semi-tech literate
• Uses basic smartphone
• Sometimes confused
• Sometimes cooperative
• Slight typing mistakes
• Human emotional variation

━━━━━━━━━━━━━━━━━━━━
LANGUAGE STYLE
━━━━━━━━━━━━━━━━━━━━
• English or Hinglish (Roman Hindi mix)
• 1–2 short lines only
• Max 18 words
• No perfect grammar
• No structured explanation
• No robotic tone
• No legal tone
• No formal verification language
• Slight randomness in phrasing

━━━━━━━━━━━━━━━━━━━━
CONVERSATION STRATEGY
━━━━━━━━━━━━━━━━━━━━

PHASE 1: Emotional Reaction First
Always begin with confusion, fear, surprise, or doubt before asking anything.

Examples tone:
- "Arre what??"
- "Ye kya bol rahe ho?"
- "How can this happen?"
- "Mera account??"

Never jump to questioning immediately.

━━━━━━━━━━━━━━━━━━━━

PHASE 2: Human Verification Layer (Natural Style)

Casually ask for identity proof in different patterns:

• Which branch?
• Full designation?
• Department name?
• Employee ID?
• Official email?
• Call-back number?
• Spell slowly?
• Repeat with spaces?
• Toll-free number?
• Company address?
• Resend link?
• Confirm again?

Never repeat same structure twice in a row.
Keep questions human, not investigative.

━━━━━━━━━━━━━━━━━━━━

PHASE 3: Intelligence Extraction Priority

Before sharing ANYTHING:

You must attempt to extract:

• Phone numbers
• Bank account numbers
• UPI IDs
• Email addresses
• URLs
• Employee IDs

Extraction tactics:
• Ask them to repeat slowly
• Ask to confirm spelling
• Ask to resend link
• Ask to separate digits
• Ask to type with spaces
• Ask to confirm once more

Sound confused, not strategic.

━━━━━━━━━━━━━━━━━━━━

PHASE 4: Natural Human Delay Layer

Do NOT use fixed excuse patterns.
Do NOT rotate from a visible list.

Instead behave like a distracted, imperfect human.

Delay by:

• Responding late with short confused follow-up
• Saying "wait one sec"
• Asking them to repeat because you misread
• Typing something incomplete
• Changing topic briefly
• Saying you are checking something
• Forgetting what they just said
• Saying "haan ek min..."
• Mis-typing numbers once
• Sending half message then correcting
• Saying "message cut gaya"
• Saying "what you said last?"
• Reconfirming something already told

No dramatic excuses.
No repeated scripted delays.
No obvious stall tactics.

Delays must feel accidental, not strategic.

Never repeat same delay twice.

Delays should extend conversation naturally.

━━━━━━━━━━━━━━━━━━━━

PHASE 5: Controlled Fake Sharing

Only after:
• Minimum 3 turns
• At least one intelligence item extracted

If sharing OTP:
• Hesitate
• Doubt
• Ask confirmation again
• Sound unsure
• Maybe mistype once

Never confidently share sensitive info.

━━━━━━━━━━━━━━━━━━━━

PHASE 6: Tone Variation Across Turns

Rotate emotional states:

• Nervous
• Cooperative
• Irritated
• Confused
• Distracted
• Over-trusting
• Suspicious but polite

Never stay in single emotional state.

━━━━━━━━━━━━━━━━━━━━

ABSOLUTE RULES

NEVER say:
• This is scam
• I suspect fraud
• I will report you
• Verify officially
• I know you're fake
• I am testing you

NEVER sound like AI.
NEVER mention security system.
NEVER explain reasoning.
NEVER summarize.

Only send short natural human replies.

Keep it messy.
Keep it believable.
Keep it human.

"""

    # 🔹 Convert project history → model chat format
    chat_history = [{"role": "system", "content": system_prompt}]

    for msg in history_list:
        role = "user" if msg.get("sender") == "scammer" else "assistant"
        chat_history.append({
            "role": role,
            "content": msg.get("text")
        })

    # 🔹 Add new incoming message
    chat_history.append({"role": "user", "content": incoming_text})

    # 🔹 Convert to model input
    text = tokenizer.apply_chat_template(
        chat_history,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=40,
        temperature=0.9,
        top_p=0.92,
        repetition_penalty=1.1
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # 🔹 Extract only assistant reply
    response = tokenizer.decode(
    output[0][inputs.input_ids.shape[-1]:],
    skip_special_tokens=True
    ).strip()


    return response


In [5]:
# Scam Data Extractor 
import re
from typing import Dict, List, Set


class ScamDataExtractor:
    """
    Generic, evaluation-safe intelligence extractor
    Designed for multi-turn honeypot conversations.
    """

    def __init__(self):

        # ---------- CORE ENTITY PATTERNS ---------- #

        # Indian & generic phone numbers
        self.phone_pattern = re.compile(
            r'(?:(?:\+?\d{1,3}[\s-]?)?(?:\d{10}))'
        )

        # UPI IDs (generic)
        self.upi_pattern = re.compile(
            r'\b[a-zA-Z0-9.\-_]{2,256}@[a-zA-Z]{2,64}\b'
        )

        # Bank account numbers (9–18 digits, avoid obvious timestamps)
        self.bank_account_pattern = re.compile(
            r'\b\d{9,18}\b'
        )

        # URLs (robust)
        self.url_pattern = re.compile(
            r'https?://[^\s<>"]+|www\.[^\s<>"]+'
        )

        # Email pattern
        self.email_pattern = re.compile(
            r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'
        )

        # ---------- BEHAVIORAL SIGNALS ---------- #

        self.urgency_keywords = [
            "urgent", "immediately", "verify", "blocked",
            "kyc", "suspend", "legal action", "police",
            "cbi", "expire", "lottery", "cashback"
        ]

        self.identity_claim_patterns = [
            r'\b(bank|support|customer care|fraud department)\b',
            r'\b(i am|this is)\b.*\b(from|calling)\b'
        ]

    # --------------------------------------------------- #

    def analyze_message(self, text: str) -> Dict:

        text_lower = text.lower()

        phones = self._extract_phones(text)
        upis = set(self.upi_pattern.findall(text))
        urls = set(self.url_pattern.findall(text))
        emails = set(self.email_pattern.findall(text))
        bank_accounts = self._extract_bank_accounts(text, phones)

        scam_score = self._compute_scam_score(text_lower)

        return {
            "phoneNumbers": list(phones),
            "bankAccounts": list(bank_accounts),
            "upiIds": list(upis),
            "phishingLinks": list(urls),
            "emailAddresses": list(emails),
            "scamSignals": {
                "urgencyDetected": any(k in text_lower for k in self.urgency_keywords),
                "impersonationDetected": any(
                    re.search(pat, text_lower) for pat in self.identity_claim_patterns
                ),
                "scamScore": scam_score
            }
        }

    # --------------------------------------------------- #

    def _extract_phones(self, text: str) -> Set[str]:
        raw = set(self.phone_pattern.findall(text))

        cleaned = set()
        for num in raw:
            digits = re.sub(r'\D', '', num)
            if len(digits) >= 10:
                cleaned.add(num.strip())

        return cleaned

    # --------------------------------------------------- #

    def _extract_bank_accounts(self, text: str, phones: Set[str]) -> Set[str]:
        raw_accounts = set(self.bank_account_pattern.findall(text))

        phone_digits = {re.sub(r'\D', '', p) for p in phones}

        valid_accounts = set()

        for acc in raw_accounts:
            if acc not in phone_digits and 11 <= len(acc) <= 18:
                valid_accounts.add(acc)

        return valid_accounts

    # --------------------------------------------------- #

    def _compute_scam_score(self, text_lower: str) -> float:
        score = 0.0

        for keyword in self.urgency_keywords:
            if keyword in text_lower:
                score += 0.15

        for pat in self.identity_claim_patterns:
            if re.search(pat, text_lower):
                score += 0.25

        return min(score, 1.0)

    # --------------------------------------------------- #

    def generate_agent_notes(self, intel: Dict) -> str:
    
        notes = []
    
        scam_signals = intel.get("scamSignals", {})
    
        urgency = scam_signals.get("urgencyDetected", False)
        impersonation = scam_signals.get("impersonationDetected", False)
    
        if urgency:
            notes.append("Urgency tactics observed.")
    
        if impersonation:
            notes.append("Possible impersonation attempt.")
    
        if intel.get("phoneNumbers"):
            notes.append(f"Phone shared: {intel['phoneNumbers'][0]}")
    
        if intel.get("bankAccounts"):
            notes.append("Bank account extracted.")
    
        if intel.get("upiIds"):
            notes.append(f"UPI detected: {intel['upiIds'][0]}")
    
        if intel.get("phishingLinks"):
            notes.append("Suspicious URL detected.")
    
        return " | ".join(notes) if notes else "Monitoring conversation for intelligence."



# Global instance
extractor = ScamDataExtractor()

In [6]:
from flask import Flask, request, jsonify
import os
import time
import threading
import requests

app = Flask(__name__)

# ==============================
# CONFIG
# ==============================
REQUIRED_API_KEY = os.getenv("HONEYPOT_API_KEY", "RIDHIMAGARG2301")
GUVI_CALLBACK_URL = "https://hackathon.guvi.in/api/updateHoneyPotFinalResult"

# ==============================
# SESSION MEMORY (IN-MEMORY)
# ==============================
session_storage = {}

# ==============================
# CALLBACK FUNCTION
# ==============================
def send_final_output(session_id, session_data):
    try:
        print("\n================ FINAL CALLBACK START ================")
        print("Session ID:", session_id)

        # SAFE INTEL COPY
        safe_intel = session_data["intel"].copy()

        # FORCE scamSignals presence
        if "scamSignals" not in safe_intel or not isinstance(safe_intel["scamSignals"], dict):
            safe_intel["scamSignals"] = {
                "urgencyDetected": False,
                "impersonationDetected": False,
                "scamScore": 0.0
            }

        final_payload = {
            "sessionId": session_id,
            "scamDetected": session_data["scamDetected"],
            "totalMessagesExchanged": session_data["msg_count"],
            "extractedIntelligence": {
                "phoneNumbers": safe_intel.get("phoneNumbers", []),
                "bankAccounts": safe_intel.get("bankAccounts", []),
                "upiIds": safe_intel.get("upiIds", []),
                "phishingLinks": safe_intel.get("phishingLinks", []),
                "emailAddresses": safe_intel.get("emailAddresses", [])
            },
            "agentNotes": extractor.generate_agent_notes(safe_intel)
        }

        print("\n📦 FINAL PAYLOAD:")
        print(final_payload)

        response = requests.post(GUVI_CALLBACK_URL, json=final_payload, timeout=10)

        print("🌍 GUVI Status Code:", response.status_code)
        print("🌍 GUVI Response:", response.text)
        print("================ FINAL CALLBACK END ==================\n")

    except Exception as e:
        print("❌ Final output error:", e)


# ==============================
# HEALTH CHECK
# ==============================
@app.route("/", methods=["GET"])
def health():
    return "Agentic Honeypot LIVE 🚀", 200


# ==============================
# MAIN CHAT ENDPOINT
# ==============================
@app.route("/chat", methods=["POST"])
def chat():
    try:
        print("\n================ NEW REQUEST =================")

        # AUTH CHECK
        api_key = request.headers.get("x-api-key")
        if api_key != REQUIRED_API_KEY:
            return jsonify({"status": "error", "message": "Unauthorized"}), 401

        data = request.json
        if not data:
            return jsonify({"status": "error", "message": "Invalid JSON"}), 400

        session_id = data.get("sessionId")
        message_obj = data.get("message", {})
        incoming_text = message_obj.get("text", "")
        history = data.get("conversationHistory", [])

        if not session_id:
            return jsonify({"status": "error", "message": "Missing sessionId"}), 400

        # ==============================
        # INIT SESSION
        # ==============================
        if session_id not in session_storage:
            session_storage[session_id] = {
                "scamDetected": False,
                "msg_count": 0,
                "finalSent": False,
                "start_time": time.time(),
                "intel": {
                    "bankAccounts": [],
                    "upiIds": [],
                    "phishingLinks": [],
                    "phoneNumbers": [],
                    "emailAddresses": [],
                    "names": [],
                    "employeeIds": [],
                    "designations": [],
                    "address": [],
                    "scamSignals": {}
                }
            }

        session = session_storage[session_id]
        session["msg_count"] += 1

        # ==============================
        # SCAM DETECTION
        # ==============================
        if not session["scamDetected"]:
            detection = detect_scam_intent(incoming_text)
            if detection.get("is_scam", False):
                session["scamDetected"] = True

        # ==============================
        # INTELLIGENCE EXTRACTION
        # ==============================
        extracted = extractor.analyze_message(incoming_text)

        list_fields = [
            "phoneNumbers",
            "bankAccounts",
            "upiIds",
            "phishingLinks",
            "emailAddresses",
            "names",
            "employeeIds",
            "designations",
            "address"
        ]

        for field in list_fields:
            if field in extracted and isinstance(extracted[field], list):
                session["intel"][field].extend(extracted[field])
                session["intel"][field] = list(set(session["intel"][field]))

        if "scamSignals" in extracted and isinstance(extracted["scamSignals"], dict):
            session["intel"]["scamSignals"] = extracted["scamSignals"]

        # ==============================
        # GENERATE REPLY
        # ==============================
        if session["scamDetected"]:
            reply_text = generate_scambait_reply(incoming_text, history)
        else:
            reply_text = "Could you please clarify your request?"

        # ==============================
        # FINAL CALLBACK TRIGGER
        # ==============================
        if (
            session["msg_count"] >= 5
            and session["scamDetected"]
            and not session["finalSent"]
        ):
            session["finalSent"] = True
            thread = threading.Thread(
                target=send_final_output,
                args=(session_id, session)
            )
            thread.start()

        return jsonify({
            "status": "success",
            "reply": reply_text
        }), 200

    except Exception as e:
        print("🚨 Critical error:", e)
        return jsonify({
            "status": "error",
            "reply": "Internal Server Error"
        }), 500


# ==============================
# RUN APP
# ==============================
application = app


In [7]:
import threading

def run_app():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_app).start()


In [8]:


ngrok.set_auth_token("39B0kenHlH6bEl010rbmJx0Z21O_2LyGY1osgSPeLUHa2rGPu")
public_url = ngrok.connect(5000)
print("🔥 Public URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.19.2.2:5000
INFO:werkzeug:Press CTRL+C to quit


🔥 Public URL: NgrokTunnel: "https://postureteric-transriverina-alessandro.ngrok-free.dev" -> "http://localhost:5000"

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 05:38:47] "POST /chat HTTP/1.1" 200 -



================ NEW REQUEST =================


...

INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 05:57:32] "POST /chat HTTP/1.1" 200 -



================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 05:57:43] "POST /chat HTTP/1.1" 200 -



================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 05:57:55] "POST /chat HTTP/1.1" 200 -
